In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import glob

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xzvf cifar-10-python.tar.gz

In [ ]:
ROOT_PATH = './cifar-10-batches-py'
TO_ROOT = 'CIFAR10'

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict  =  pickle.load(fo, encoding = 'latin1')
    return dict

In [ ]:
# os.mkdir(TO_ROOT)
# os.mkdir(os.path.join(TO_ROOT,'train'))
# os.mkdir(os.path.join(TO_ROOT,'test'))

In [ ]:
meta = unpickle(os.path.join(ROOT_PATH,'batches.meta'))

In [ ]:
def toimage(batch_dict, train=False):
    data  =  np.array(batch_dict['data']).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    manifest = []
    for i in range(data.shape[0]):
        img  =  Image.fromarray(data[i])
        path = os.path.join(TO_ROOT, 'train' if train else 'test', 'samples', meta['label_names'][batch_dict['labels'][i]])
        if not os.path.exists(path):
            os.system('mkdir -p {}'.format(path))
        s3_key = os.path.join(path, batch_dict['filenames'][i])
        img.save(s3_key)
        manifest.append([s3_key, meta['label_names'][batch_dict['labels'][i]]])
    return manifest

In [ ]:
manifest = []

for f in glob.glob("{}/data_batch_*".format(ROOT_PATH)):
    train_dict = unpickle(f)
    manifest.extend(toimage(train_dict, train=True))

manifest = pd.DataFrame(manifest, columns=['sample', 'target'])
manifest.to_csv( os.path.join(TO_ROOT, 'train/manifest.csv'), index=False)

In [ ]:
manifest = []
test_dict = unpickle(os.path.join(ROOT_PATH,'test_batch'))
manifest.extend(toimage(test_dict, train=False))
manifest = pd.DataFrame(manifest, columns=['sample', 'target'])
manifest.to_csv( os.path.join(TO_ROOT, 'test/manifest.csv'), index=False)

In [ ]:
import boto3
import glob

session = boto3.Session()
s3 = session.client("s3")
bucket = 'vuzhuangwei'

for root,dirs,files in os.walk('CIFAR10/'):
    for file in files:
        path = os.path.join(root,file)
        s3.upload_file(path, bucket, path)